In [24]:
!pip install scikit_posthocs -q
!pip install deepsig -q


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
import sqlite3
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from ana_utils import convert_listlike_cols, db_path, font_size, font_size_large, df_to_latex_table
import json
from ana_utils import *
import numpy as np
from deepsig import aso
from collections import defaultdict
from tqdm.auto import tqdm

In [2]:
METRIC = "r2_score"

# DeepSig
debug = True
seed = 1234
np.random.seed(seed)
min_eps_thr = 0.2
if debug:
    n_sig = 5
else:
    n_sig = 1000

In [3]:
def sec2str(seconds):
    if seconds < 60:
        return str(seconds) + "s"
    elif seconds < 60*60:
        return str(seconds // 60) + "min"
    else:
        return str(seconds // (60*60)) + "h"

conn_ctr = sqlite3.connect(ctr_path)
#df = pd.read_sql(f"SELECT model, search_space_hash, {METRIC}, data_config_hash, data_id, search_space_name FROM results ", conn)
df_ctr = pd.read_sql(f"SELECT results.model as model, search_space_name as Model, {METRIC}, data_config_hash, data_id, json_extract(search_spaces.config, '$.fit_kwargs') as fit_kwargs FROM results join search_spaces on results.search_space_hash == search_spaces.hash_key", conn_ctr)
conn_ctr.close()
df_ctr = convert_listlike_cols(df_ctr)


conn_vimet = sqlite3.connect(vimet_path)
df_vimet = pd.read_sql(f"SELECT results.model as model, search_space_name as Model, {METRIC}, data_config_hash, data_id, json_extract(search_spaces.config, '$.fit_kwargs') as fit_kwargs FROM results join search_spaces on results.search_space_hash == search_spaces.hash_key", conn_vimet)
conn_vimet.close()
df_vimet = convert_listlike_cols(df_vimet)
#list(df[df["model"]=="AutoGluon"]["config"])[0]
# df["Model"] = df[["model", "fit_kwargs"]].apply(lambda x: x["model"] if x["fit_kwargs"] is None else f"{x['model']} ({sec2str(json.loads(x['fit_kwargs'])['time_limit'])})", axis=1)
# df = df.drop(["fit_kwargs"], axis=1)
# df[df["model"] == "AutoGluon"]
df_ctr

r2_score
r2_score


,model,Model,r2_score,data_config_hash,data_id,fit_kwargs,r2_score_corrected
0,ElasticNet,ElasticNet,"[0.10567121376356414, 0.7244679272319543, 0.49...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,24,None,"[0.10567121376356414, 0.7244679272319543, 0.49..."
1,ExtraTrees,ExtraTrees,"[-0.024123602331189042, -0.10398036220475104, ...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,30,None,"[0, 0, 0, 0.15786634070813144, 0, 0, 0, 0, 0, 0]"
2,Dummy,Dummy,"[-0.0005395067877793647, -4.1781091721926344e-...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,9,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,ElasticNet,ElasticNet,"[0.9496890647722223, 0.940504449782539, 0.9469...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,28,None,"[0.9496890647722223, 0.940504449782539, 0.9469..."
4,BayesianRidge,BayesianRidge,"[0.1053952552242644, 0.7251447692622289, 0.510...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,24,None,"[0.1053952552242644, 0.7251447692622289, 0.510..."
...,...,...,...,...,...,...,...
1137,CubistModel,CubistModel,"[None, None, None, None, None, None, None, Non...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,20,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1138,GaussianProcess,GaussianProcess,"[0.31157404729224947, None, None, None, None, ...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,12,None,"[0.31157404729224947, 0, 0, 0, 0, 0, 0.2914187..."
1139,SupportVectorRegression,SupportVectorRegression,"[0.644826767855609, 0.6495741898960192, 0.6164...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,20,None,"[0.644826767855609, 0.6495741898960192, 0.6164..."
1140,CubistModel,CubistModel,"[None, None, None, None, None, None, None, Non...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,15,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [4]:
df_ctr['Model'].value_counts()

Model
ElasticNet                 35
ExtraTrees                 35
Dummy                      35
BayesianRidge              35
DecisionTree               35
Bagging                    35
Huber                      35
AdaBoost                   35
CatBoost                   35
GradientBoosting           35
KNN                        35
Lasso                      35
PartialLeastSquares        35
LightGBM                   35
M5                         35
PassiveAggressive          35
GaussianProcess            35
RidgeRegressor             35
AutoGluon(2h)              35
AutoGluon(30min)           35
SGD                        35
MLP                        35
AutoGluon(1h)              35
AutoGluon(1min)            35
AutoGluon(10min)           35
LassoLars                  34
LARS                       34
AutoGluon(5min)            34
Tweedie                    31
SupportVectorRegression    30
XGBoost                    28
PolynomialRegression       27
CubistModel                26
Rand

In [5]:
# groups = df_ctr.groupby(by=["data_config_hash", "data_id"])
# group_collection = []
# all_models = df_ctr["Model"].unique()

# baselines = ["CatBoost", "ExtraTrees", "AutoGluon(1h)", "AutoGluon(2h)"]
# beat_baseline = {b: {m: 0 for m in all_models} for b in baselines} # baseline: {model: wins}

# for (hash, data_id), group_df in groups:
#     model_scores = group_df[["Model", METRIC]]
#     model_scores = model_scores.set_index(model_scores["Model"])
#     model_scores_dict = model_scores[METRIC].to_dict()
#     for baseline_model in baselines:
#         if baseline_model not in model_scores_dict:
#             continue
#         for model in model_scores_dict.keys():
#             min_eps = aso(model_scores_dict[model], model_scores_dict[baseline_model], seed=seed)
#             if min_eps < min_eps_thr:
#                 beat_baseline[baseline_model][model] += 1
#     # eps_min = multi_aso(model_scores_dict, confidence_level=0.95, return_df=True, seed=seed) #Too slow and convoluted but interesting
#     break
# beat_baseline

In [6]:
# Name 
# Avg Normalized Performance
# Avg Normalized Performance (Without AG) 
# Avg Rank
# Avg Rank (Without AG) ~ To answer is the weakest AG already the best method
# Avg Absolute Inference Time per sample
# sig_wins = pd.DataFrame(beat_baseline).fillna(0).astype(int)
# sig_wins = sig_wins.rename({col: f"Sig Wins against {col}" for col in sig_wins.columns}, axis=1)
# sig_wins

In [7]:
def calc_statistics(df):
    groups = df.groupby(by=["data_config_hash", "data_id"])
    group_collection = []
    all_models = df["Model"].unique()
    baselines = ["CatBoost", "ExtraTrees", "AutoGluon(1h)", "AutoGluon(2h)"]
    beat_baseline = {b: {m: 0 for m in all_models} for b in baselines} # baseline: {model: wins}
    
    for (hash, data_id), group_df in tqdm(groups):
        # Calculate Significant Wins against selected baselines
        model_scores = group_df[["Model", METRIC]]
        model_scores = model_scores.set_index(model_scores["Model"])
        model_scores_dict = model_scores[METRIC].to_dict()
        for baseline_model in tqdm(baselines, desc="DeepSig Significance Tests"):
            if baseline_model not in model_scores_dict:
                continue
            for model in model_scores_dict.keys():
                min_eps = aso(model_scores_dict[model], model_scores_dict[baseline_model], seed=seed, show_progress=False, num_bootstrap_iterations=n_sig)
                if min_eps < min_eps_thr:
                    beat_baseline[baseline_model][model] += 1

        
        # Because if worse than zero we dont really care by how much
        # group_df.loc[group_df["avg_metric"] < 0, "avg_metric"] = 0 # Obsolete as moved to outside
        
        # Set Missing Entries to 0 - they likely crahsed
        group_df = group_df.set_index(group_df["Model"])
        group_df = group_df[~group_df.index.duplicated(keep="first")]
        group_df = group_df.drop(["Model"], axis=1)
        group_df = group_df.reindex(all_models, fill_value=0)
        group_df["Model"] = group_df.index
        group_df = group_df.reset_index(drop=True)
        
        maximum = group_df["avg_metric"].max()
        median = group_df["avg_metric"].median()
        if maximum <= 0.001:
            group_df["normalized_metric"] = 0 # If all fail, all fail simple as that
        else:
            group_df["normalized_metric"] = (group_df["avg_metric"] - median) / (maximum-median)
        assert group_df["normalized_metric"].max() < 1.2
        group_df["rank"] = group_df["avg_metric"].rank(method="dense", ascending=False)
        group_df["rounded_rank"] = group_df["avg_metric"].round(2).rank(method="dense", ascending=False)
        group_df["losses(tied)"] = (group_df["rounded_rank"] == group_df["rounded_rank"].max())
        group_df["true losses(tied)"] = (group_df["rounded_rank"] == group_df["rounded_rank"].max()) & (group_df["rounded_rank"] != group_df["rounded_rank"].min())
        group_df["true losses(raw)"] = (group_df["rank"] == group_df["rank"].max()) & (group_df["rank"] != group_df["rank"].min())
        group_df["true wins(tied)"] = (group_df["rounded_rank"] == group_df["rounded_rank"].min()) & (group_df["rounded_rank"] != group_df["rounded_rank"].max())
        group_df["true wins(raw)"] = (group_df["rank"] == group_df["rank"].min()) & (group_df["rank"] != group_df["rank"].max())
    
        group_df_wo_ag = group_df[group_df["model"] != "AutoGluon"]
        maximum_wo_ag = group_df_wo_ag["avg_metric"].max()
        median_wo_ag = group_df_wo_ag["avg_metric"].median()
        if maximum_wo_ag <= 0.001:
            group_df["normalized_metric_wo_ag"] = 0 # If all fail, all fail simple as that
        else:
            group_df["normalized_metric_wo_ag"] = (group_df["avg_metric"] - median_wo_ag) / (maximum_wo_ag-median_wo_ag)
            
        #group_df["rank_wo_ag"] = group_df["avg_metric"].rank(ascending=False)
    
        gavg = (
            group_df.groupby("model", as_index=False)
              .agg(avg_metric=("avg_metric", "max"))
        )
        
        # 2) rank groups by that average
        gavg["rank_wo_ag"] = gavg["avg_metric"].rank(method="dense", ascending=False)
        gavg["rank_wo_ag_round"] = gavg["avg_metric"].round(2).rank(method="dense", ascending=False)
        rank_wo_ag_round = gavg[gavg["model"]!="AutoGluon"]["avg_metric"].round(2).rank(method="dense", ascending=False)
        gavg["victories_wo_ag(ties)"] = (rank_wo_ag_round == 1)
        gavg["victories_wo_ag"] = gavg[gavg["model"]!="AutoGluon"]["avg_metric"].rank(method="dense", ascending=False) == 1
        
        
        # 3) merge back — no restructuring of df
        group_df = group_df.merge(gavg[["model", "rank_wo_ag", "rank_wo_ag_round", "victories_wo_ag", "victories_wo_ag(ties)"]], on="model", how="left")
        #print(group_df[group_df["model"] == "AutoGluon"]["rank_wo_ag"])
        if list(group_df[group_df["model"] == "AutoGluon"]["rank_wo_ag_round"])[0] > 1:
            print(maximum, gavg[gavg["model"] == "AutoGluon"][["rank_wo_ag_round", "avg_metric"]])
        
        autogluon_names = list(group_df[group_df["model"]=="AutoGluon"]["Model"])
        contestants = autogluon_names + ["CatBoost", "ExtraTrees"]
        ag_max_perf = 0
        for contestant in contestants:
            ag_perf = group_df[group_df["Model"] == contestant]["avg_metric"].iloc[0]
            group_df[f"victories_against_{contestant}"] = group_df["avg_metric"] > ag_perf
            group_df[f"major_victories_against_{contestant}"] = group_df["avg_metric"] > (ag_perf*1.01)
            if ag_perf > ag_max_perf:
                ag_max_perf = ag_perf
        group_df[f"victories_against_best_ag"] = group_df["avg_metric"] > ag_max_perf
        group_df[f"major_victories_against_best_ag"] = group_df["avg_metric"] > (ag_max_perf*1.01)
        
    
        group_df["failures"] = (group_df["avg_metric"] <= 0.001) | group_df["avg_metric"].isna()
        group_df = group_df.reset_index(drop=False)
        # TODO Victories against AG (x/y/z/W)
        # TODO Victories without AG
        # best_models = list(group_df[group_df["avg_metric"] >= maximum*0.999]["model"])
        # if not "AutoGluon" in best_models:
            
        #     print(maximum, best_models)
        #     print(group_df[group_df["model"] == "AutoGluon"]["avg_metric"])
        group_collection.append(group_df)
    df_ = pd.concat(group_collection)
    
    df_ = df_.groupby(by=["Model"]).agg(
        Avg_R2 = ("avg_metric", "mean"),
        Rank = ("rank", "mean"),
        Rank_round = ("rounded_rank", "mean"),
        Rank_wo_AG = ("rank_wo_ag", "mean"), # Without counting AG several times
        Rank_wo_AG_round = ("rank_wo_ag_round", "mean"),
        Norm_R2 = ("normalized_metric", "mean"),
        Norm_R2_min=("normalized_metric", "min"),
        Norm_R2_max=("normalized_metric", "max"),
        Norm_R2_wo_AG = ("normalized_metric_wo_ag", "mean"),
        Norm_R2_wo_AG_min = ("normalized_metric_wo_ag", "min"),
        Norm_R2_wo_AG_max = ("normalized_metric_wo_ag", "max"),
        Wins_wo_AG = ("victories_wo_ag", "sum"),
        Wins_Against_AG_5min = ("victories_against_AutoGluon(5min)", "sum"),
        Wins_Against_AG_30min = ("victories_against_AutoGluon(30min)", "sum"),
        Wins_Against_AG_1h = ("victories_against_AutoGluon(1h)", "sum"),
        Wins_Against_AG_2h = ("victories_against_AutoGluon(2h)", "sum"),
        Major_Wins_Against_AG_1h = ("major_victories_against_AutoGluon(1h)", "sum"),
        Wins_Against_AG_best = ("victories_against_best_ag", "sum"),
        Major_Wins_Against_AG_best = ("major_victories_against_best_ag", "sum"),
        Wins_Against_CatBoost = ("victories_against_CatBoost", "sum"),
        Wins_Against_ExtraTrees = ("victories_against_ExtraTrees", "sum"),
        Losses_tied = ("losses(tied)", "sum"),
        True_Losses_tied = ("true losses(tied)", "sum"),
        True_Losses_raw = ("true losses(raw)", "sum"),
        True_Wins_tied = ("true wins(tied)", "sum"),
        True_Wins_raw = ("true wins(raw)", "sum"),
        failures = ("failures", "sum"),
    )

    sig_wins = pd.DataFrame(beat_baseline).fillna(0).astype(int)
    sig_wins = sig_wins.rename({col: f"Sig Wins against {col}" for col in sig_wins.columns}, axis=1)
    df_ = df_.join(sig_wins)
    print(group_df[METRIC])
    return df_.sort_values(by=["True_Wins_tied", "Rank"])

In [8]:
# TODO Create Second Version with uncorrected results for appendix
df_ctr[METRIC] = df_ctr[METRIC+"_corrected"] # Comment out
df_ctr["avg_metric"] = df_ctr[METRIC].apply(
    lambda xs: np.nanmean([x if x is not None else np.nan for x in xs])
)
# df_ctr["avg_metric"] = df_ctr[METRIC+"_corrected"].apply(
#     lambda x: sum(x) / len(x) if x else None
# )
df_ctr_ready = calc_statistics(df_ctr)
df_ctr_ready

  0%|          | 0/35 [00:00<?, ?it/s]

DeepSig Significance Tests:   0%|          | 0/4 [00:00<?, ?it/s]

/home/jipo020b/.local/lib/python3.10/site-packages/deepsig/aso.py:390: UserWarning: Division by zero encountered in violation ratio.
  warn("Division by zero encountered in violation ratio.")


DeepSig Significance Tests:   0%|          | 0/4 [00:00<?, ?it/s]

/home/jipo020b/.local/lib/python3.10/site-packages/deepsig/aso.py:390: UserWarning: Division by zero encountered in violation ratio.
  warn("Division by zero encountered in violation ratio.")


...

/home/jipo020b/.local/lib/python3.10/site-packages/deepsig/aso.py:390: UserWarning: Division by zero encountered in violation ratio.
  warn("Division by zero encountered in violation ratio.")


DeepSig Significance Tests:   0%|          | 0/4 [00:00<?, ?it/s]

/home/jipo020b/.local/lib/python3.10/site-packages/deepsig/aso.py:390: UserWarning: Division by zero encountered in violation ratio.
  warn("Division by zero encountered in violation ratio.")


0     [0.5793805469852513, 0.5453118862684447, 0.549...
1     [0.6733920150020394, 0.5979700933376837, 0.680...
2                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3     [0.5792544481289225, 0.5453003021882409, 0.547...
4     [0.5784533260898252, 0.47610974362792224, 0.52...
5     [0.27275993231209217, 0.4031192539418823, 0.43...
6                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7     [0.6630774365524876, 0.6114265505760741, 0.695...
8     [0.7210061238885122, 0.6421096416935099, 0.686...
9     [0.6926871390811642, 0.6448541529622249, 0.658...
10    [0.5461906343060163, 0.5287785497113879, 0.511...
11    [0.5797947412481897, 0.5457668091022094, 0.544...
12    [0.5784334053940771, 0.5453299970526431, 0.549...
13    [0.5781572107491826, 0.5453795931097871, 0.554...
14    [0.717711922235138, 0.6715922253658309, 0.7078...
15    [0.5767595560866273, 0.5374384668351385, 0.515...
16    [0.6895286222413359, 0.6671412276007342, 0.702...
17                       [0, 0, 0, 0, 0, 0, 0, 0

,Avg_R2,Rank,Rank_round,Rank_wo_AG,Rank_wo_AG_round,Norm_R2,Norm_R2_min,Norm_R2_max,Norm_R2_wo_AG,Norm_R2_wo_AG_min,...,Losses_tied,True_Losses_tied,True_Losses_raw,True_Wins_tied,True_Wins_raw,failures,Sig Wins against CatBoost,Sig Wins against ExtraTrees,Sig Wins against AutoGluon(1h),Sig Wins against AutoGluon(2h)
Model,,,,,,,,,,,,,,,,,,,,,
GaussianProcess,0.572964,18.314286,7.400000,13.514286,6.600000,-0.196235,-3.523476,0.884554,0.171407,-2.810111,...,1,1,1,0,0,1,2,6,0,0
Bagging,0.563148,20.228571,8.428571,15.485714,7.657143,-0.726729,-10.783665,0.881505,-0.188854,-5.260566,...,0,0,0,0,0,0,0,2,0,0
KNN,0.532439,20.371429,8.600000,15.514286,7.800000,-1.238537,-18.790833,0.702914,-0.570672,-14.936601,...,1,1,1,0,0,1,0,2,0,0
MLP,0.453703,22.114286,9.714286,17.342857,8.942857,-1.781606,-15.821298,0.945555,-0.820366,-11.443401,...,5,5,2,0,0,3,2,1,1,1
Tweedie,0.459779,22.257143,9.257143,17.485714,8.457143,-2.465195,-51.733546,0.604425,-1.022140,-27.016907,...,5,5,5,0,0,5,1,1,0,0
SupportVectorRegression,0.239358,24.771429,11.085714,19.914286,10.314286,-5.589203,-51.733546,0.874375,-2.854367,-27.016907,...,18,18,17,0,0,17,3,2,0,0
CubistModel,0.000000,30.628571,14.514286,25.742857,13.714286,-10.275809,-69.736546,-0.031089,-5.607899,-33.045158,...,35,35,35,0,0,35,0,0,0,0
Dummy,0.000000,30.628571,14.514286,25.742857,13.714286,-10.275809,-69.736546,-0.031089,-5.607899,-33.045158,...,35,35,35,0,0,35,0,0,0,0
ElasticNet,0.528967,19.828571,8.371429,15.400000,7.600000,-0.551740,-3.192783,1.000000,0.075443,-0.576611,...,1,1,0,1,0,0,5,3,2,3


In [11]:
def bold_if(value, best_value):
    if isinstance(value, int):
        if value == best_value:
            return f"\\textbf{{{value}}}"
        else:
            return f"{value}"
    else: 
        if value == best_value:
            return f"\\textbf{{{value:.2f}}}"
        else:
            return f"{value:.2f}"

def pretty(x, value_name):
    return bold_if(x[value_name], best_entries[value_name])

best_entries = {
    "Norm_R2_min": df_ctr_ready["Norm_R2_min"].max(),
    "Norm_R2_max": df_ctr_ready["Norm_R2_max"].max(),
    "Norm_R2": df_ctr_ready["Norm_R2"].max(),
    "Norm_R2_wo_AG_min": df_ctr_ready["Norm_R2_wo_AG_min"].max(),
    "Norm_R2_wo_AG_max": df_ctr_ready["Norm_R2_wo_AG_max"].max(),
    "Norm_R2_wo_AG": df_ctr_ready["Norm_R2_wo_AG"].max(),
    "Rank": df_ctr_ready["Rank"].min(),
    "Rank_round": df_ctr_ready["Rank_round"].min(),
    "True_Losses_tied": df_ctr_ready["True_Losses_tied"].min(),
    "True_Losses_raw": df_ctr_ready["True_Losses_raw"].min(),
    "True_Wins_tied": df_ctr_ready["True_Wins_tied"].max(),
    "True_Wins_raw": df_ctr_ready["True_Wins_raw"].max(),
    "failures": df_ctr_ready["failures"].min(),
    "Wins_Against_AG_1h": df_ctr_ready["Wins_Against_AG_1h"].max(),
    "Wins_Against_AG_2h": df_ctr_ready["Wins_Against_AG_2h"].max(),
    "Wins_Against_CatBoost": df_ctr_ready["Wins_Against_CatBoost"].max(),
    "Wins_Against_ExtraTrees": df_ctr_ready["Wins_Against_ExtraTrees"].max(),
    "Sig Wins against AutoGluon(1h)": df_ctr_ready["Sig Wins against AutoGluon(1h)"].max(),
    "Sig Wins against AutoGluon(2h)": df_ctr_ready["Sig Wins against AutoGluon(2h)"].max(),
    "Sig Wins against CatBoost": df_ctr_ready["Sig Wins against CatBoost"].max(),
    "Sig Wins against ExtraTrees": df_ctr_ready["Sig Wins against ExtraTrees"].max(),
}
df_ctr_ready["Norm R2"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Norm_R2_min')}/{pretty(x, 'Norm_R2')}/{pretty(x, 'Norm_R2_max')}", axis=1)
df_ctr_ready["Norm R2 (wo AG)"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Norm_R2_wo_AG_min')}/{pretty(x, 'Norm_R2_wo_AG')}/{pretty(x, 'Norm_R2_wo_AG_max')}", axis=1)
df_ctr_ready["Avg Rank"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Rank')} ({pretty(x, 'Rank_round')})", axis=1)
df_ctr_ready["Wins"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'True_Wins_tied')} ({pretty(x, 'True_Wins_raw')})", axis=1) # Based on rounded Rank + Ignore Wins when EVERY model performs the same
df_ctr_ready["Losses"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'True_Losses_tied')} ({pretty(x, 'True_Losses_raw')})", axis=1) # Based on rounded Rank + Ignore Losses when EVERY model performs terrible
df_ctr_ready["Failures"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'failures')}", axis=1) 

df_ctr_ready["Wins (AG 1h)"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Wins_Against_AG_1h')} ({pretty(x, 'Sig Wins against AutoGluon(1h)')})", axis=1) 
df_ctr_ready["Wins (AG 2h)"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Wins_Against_AG_2h')} ({pretty(x, 'Sig Wins against AutoGluon(2h)')})", axis=1) 
df_ctr_ready["Wins (CatBoost)"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Wins_Against_CatBoost')} ({pretty(x, 'Sig Wins against CatBoost')})", axis=1) 
df_ctr_ready["Wins (ExtraTrees)"] = df_ctr_ready.apply(lambda x: f"{pretty(x, 'Wins_Against_ExtraTrees')} ({pretty(x, 'Sig Wins against ExtraTrees')})", axis=1) 

df_for_render = df_ctr_ready[["Norm R2", "Norm R2 (wo AG)", "Avg Rank", "Wins", "Losses", "Failures", "Wins (AG 1h)", "Wins (CatBoost)", "Wins (ExtraTrees)"]]
df_for_render

,Norm R2,Norm R2 (wo AG),Avg Rank,Wins,Losses,Failures,Wins (AG 1h),Wins (CatBoost),Wins (ExtraTrees)
Model,,,,,,,,,
GaussianProcess,-3.52/-0.20/0.88,-2.81/0.17/1.00,18.31 (7.40),0 (0),1 (1),1,0 (0),6 (2),8 (6)
Bagging,-10.78/-0.73/0.88,-5.26/-0.19/0.96,20.23 (8.43),0 (0),\textbf{0} (\textbf{0}),\textbf{0},1 (0),2 (0),2 (2)
KNN,-18.79/-1.24/0.70,-14.94/-0.57/0.86,20.37 (8.60),0 (0),1 (1),1,0 (0),3 (0),3 (2)
MLP,-15.82/-1.78/0.95,-11.44/-0.82/1.00,22.11 (9.71),0 (0),5 (2),3,1 (1),4 (2),4 (1)
Tweedie,-51.73/-2.47/0.60,-27.02/-1.02/0.92,22.26 (9.26),0 (0),5 (5),5,0 (0),2 (1),1 (1)
SupportVectorRegression,-51.73/-5.59/0.87,-27.02/-2.85/0.98,24.77 (11.09),0 (0),18 (17),17,0 (0),4 (3),2 (2)
CubistModel,-69.74/-10.28/-0.03,-33.05/-5.61/-0.06,30.63 (14.51),0 (0),35 (35),35,0 (0),0 (0),0 (0)
Dummy,-69.74/-10.28/-0.03,-33.05/-5.61/-0.06,30.63 (14.51),0 (0),35 (35),35,0 (0),0 (0),0 (0)
ElasticNet,-3.19/-0.55/1.00,-0.58/0.08/1.00,19.83 (8.37),1 (0),1 (\textbf{0}),\textbf{0},3 (2),6 (5),3 (3)


In [ ]:
combined = pd.concat(
    {"ctr": df_for_render, "vimet": df_for_render}, # TODO Add ViMet
    axis=1
)

# Optional: put metric first, source second -> col -> (ctr/vimet)
combined = combined.swaplevel(0, 1, axis=1).sort_index(axis=1)
combined

In [10]:
df_["Model"] = df_.index

df_to_latex_table(
    df=df_[["Model", "normalized_metric_wo_ag", "normalized_metric", "rounded_rank", "true losses(tied)", "failures"]],
    filepath="./cache/ranking.tex",
    caption="Performance comparison of all evaluated methods against AutoGluon",
    label="tab:ranking",
    float_fmt=".2f",
)

'\\begin{table}[t]\n\\centering\n\\begin{tabular}{lrrrrr}\n\\toprule\n Model                   &   normalized\\_metric\\_wo\\_ag &   normalized\\_metric &   rounded\\_rank &   true losses(tied) &   failures \\\\\n\\midrule\n AdaBoost                &                      0.36 &                0.17 &           5.4  &                   3 &          4 \\\\\n AutoGluon (10min)       &                      0.92 &                0.82 &           1.66 &                   0 &          1 \\\\\n AutoGluon (1h)          &                      1    &                0.91 &           1.51 &                   0 &          1 \\\\\n AutoGluon (1min)        &                      0.21 &                0.07 &           2.26 &                   0 &          1 \\\\\n AutoGluon (30min)       &                      1    &                0.91 &           1.49 &                   0 &          1 \\\\\n AutoGluon (4h)          &                      0.95 &                0.85 &           1.66 &                  

In [8]:
# TODO 
# Get Names of outlier DS describe what happened (fair but minor, fair server but only two models better, all shit)

In [13]:
import scikit_posthocs as sp
import matplotlib.pyplot as plt

fig = sp.critical_difference_diagram(
    df.values,
    df.columns.tolist(),
    alpha=0.05,  # Nemenyi
)
plt.show()


TypeError: critical_difference_diagram() got an unexpected keyword argument 'alpha'